In [1]:
# Dependencies
from flask import Flask, render_template, redirect, Markup, jsonify
from flask_pymongo import PyMongo
from bson.json_util import dumps
import pymongo
from flask_cors import CORS
import pandas as pd
from pandas import read_csv
import json

In [2]:
app = Flask(__name__)

CORS(app, support_credentials = True)

# Use flask_pymongo to set up mongo connection
app.config["MONGO_URI"] = "mongodb://localhost:27017/election2020"
mongo = PyMongo(app)

In [3]:
# create route that rendrs index.html template
# @app.route("/")
def index():
    
    
    state_ids = {"Alabama":"01",
                 "Alaska":"02",
                 "Arizona":"04",
                 "Arkansas":"05",
                 "California":"06",
                 "Colorado":"08",
                 "Connecticut": "09",
                 "Delaware":"10",
                 "District of Columbia":"11",
                 "Florida":"12",
                 "Georgia":"13",
                 "Hawaii":"15",
                 "Idaho": "16",
                 "Illinois": "17",
                 "Indiana": "18",
                 "Iowa": "19",
                 "Kansas":"20",
                 "Kentucky":"21",
                 "Louisiana":"22",
                 "Maine":"23",
                 "Maryland":"24" ,
                 "Massachusetts":"25",
                 "Michigan":"26",
                 "Minnesota":"27",
                 "Mississippi":"28",
                 "Missouri": "29",
                 "Montana":"30",
                 "Nebraska":"31",
                 "Nevada": "32",
                 "New Hampshire":"33",
                 "New Jersey": "34",
                 "New Mexico": "35",
                "New York":"36",
                "North Carolina": "37",
                "North Dakota": "38",
                "Ohio":"39",
                "Oklahoma": "40",
                "Oregon": "41",
                "Pennsylvania": "42",
                "Rhode Island":"44",
                "South Carolina":"45",
                "South Dakota": "46",
                "Tennessee":"47",
                "Texas": "48",
                 "Utah":"49",
                 "Vermont":"50",
                 "Virginia":"51",
                 "Washington":"53",
                 "West Virginia": "54",
                 "Wisconsin":"55",
                 "Wyoming": "56"}

candidate = mongo.db.pCandidate.find()
state = mongo.db.pState.find()
county = mongo.db.pCounty.find()
countyIdsStates = mongo.db.countyIdsStates.find()

In [4]:
all_data = [candidate, state, county, countyIdsStates]

return_list = []

for dataset in all_data:

        list_cur = list(dataset)
        json_data = dumps(list_cur)
        return_list.append(json_data)
        
# Pull only candidate data
candidate_data = return_list[0]
IdsStates_data = return_list[3]

    
# Convert string of json to a json file
candidate_json = json.loads(candidate_data)
IdsStates_json = json.loads(IdsStates_data)
    
# Convert json file to data frame
df = pd.DataFrame.from_records(candidate_json)
IdsStates_df = pd.DataFrame.from_records(IdsStates_json)

cond_cand_df = df[["state", "county", "candidate", "total_votes"]]
IdsStates_df1 = IdsStates_df[['id','county','state']]
#cond_cand_df

#df.loc[(df["candidate"] == "Joe Biden") | (df["candidate"] == "Donald Trump"),:]

# Create two separate data sets containing Biden and Trump vote counts at the county level
B_candidate_df = cond_cand_df.loc[(cond_cand_df["candidate"] == "Joe Biden"),:]
T_candidate_df = cond_cand_df.loc[(cond_cand_df["candidate"] == "Donald Trump"),:]
IdsStates_df1

,id,county,state
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


In [5]:
B_candidate_df['county'] = B_candidate_df['county'].str.replace(' County', '')
T_candidate_df['county'] = T_candidate_df['county'].str.replace(' County', '')
us_state_abbrev = {
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'}
B_candidate_df['state'] = B_candidate_df['state'].map(us_state_abbrev)
T_candidate_df['state'] = T_candidate_df['state'].map(us_state_abbrev)
B_candidate_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

,state,county,candidate,total_votes
0,DE,Kent,Joe Biden,44552
4,DE,New Castle,Joe Biden,195034
9,DE,Sussex,Joe Biden,56682
12,DC,District of Columbia,Joe Biden,39041
19,DC,Ward 2,Joe Biden,29078
...,...,...,...,...
32158,AZ,Graham,Joe Biden,4034
32162,AZ,Greenlee,Joe Biden,1182
32166,AZ,La Paz,Joe Biden,2236
32169,AZ,Maricopa,Joe Biden,1040774


In [6]:
#file_one = "data/finalcounties.csv"
#finalcountiesdf = pd.read_csv(file_one)
#finalcountiesdf.columns = ['id', 'county', 'state']
#finalcountiesdf

In [7]:
Bmerge_df = pd.merge(IdsStates_df1, B_candidate_df, on=["county","state"])
Tmerge_df = pd.merge(IdsStates_df1, T_candidate_df, on=["county","state"])
Bbubbledf = Bmerge_df[["id", "total_votes"]]
Tbubbledf = Tmerge_df[["id", "total_votes"]]
B_area_dict = dict(zip(Bbubbledf.id, Bbubbledf.total_votes))
T_area_dict = dict(zip(Tbubbledf.id, Tbubbledf.total_votes))
B_area_dict
finalJson = {}
finalJson.update({"Bcountyvotecount": B_area_dict})
finalJson.update({"Tcountyvotecount": T_area_dict})
finalJson

{'Bcountyvotecount': {1001: 7503,
  1003: 24578,
  1005: 4816,
  1007: 1986,
  1009: 2640,
  1011: 3446,
  1013: 3965,
  1015: 15216,
  1017: 6365,
  1019: 1624,
  1021: 3073,
  1023: 3127,
  1025: 5755,
  1027: 1267,
  1029: 675,
  1031: 5076,
  1033: 8343,
  1035: 2966,
  1037: 1796,
  1039: 2721,
  1041: 1700,
  1043: 4478,
  1045: 5170,
  1047: 12230,
  1051: 10367,
  1053: 4918,
  1055: 11567,
  1057: 1395,
  1059: 2086,
  1061: 1595,
  1063: 3884,
  1065: 4687,
  1067: 2606,
  1069: 12917,
  1071: 3717,
  1073: 181688,
  1075: 978,
  1077: 11915,
  1079: 3562,
  1081: 27860,
  1083: 13672,
  1085: 4972,
  1087: 7108,
  1089: 87286,
  1091: 5488,
  1093: 1463,
  1095: 5943,
  1097: 79474,
  1099: 4455,
  1101: 64529,
  1103: 13234,
  1105: 3860,
  1107: 4022,
  1109: 5636,
  1111: 2203,
  1113: 11228,
  1117: 33268,
  1119: 4648,
  1121: 13138,
  1123: 5859,
  1125: 37765,
  1127: 4834,
  1129: 2258,
  1131: 4048,
  1133: 974,
  4001: 23293,
  4003: 23732,
  4005: 44698,
  4007: 8